## Vector Operations

Working with geospatial vector data typically involves manipulating collections of features - points, lines and polygons with attributes. The basic steps are the same as you would use with any type of data: load the vector data as input, do some kind of work on the data, output the data. This lab will focus on step two, 'do some kind of work on the data', specifically looking at some analysis that can be done with vector data.

There are many operations that geopanda has made easy for you. This lab will cover some of them. Refer to [geopanda documentation](http://geopandas.org/geometric_manipulations.html) to see a complete list of the operations included with geopanda. geopandas makes available all the tools for geometric manipulations in the shapely library.

Lets get started by loading some data. The data for this practice will be NYC borough shape files:
 * [NYC Borough Boundaries](https://data.cityofnewyork.us/api/geospatial/tqmj-j8zm?method=export&format=Shapefile) - See it in action [here](https://data.cityofnewyork.us/City-Government/Borough-Boundaries/tqmj-j8zm)
 * [NYC Wi-Fi Hotspot Locations](https://data.cityofnewyork.us/api/geospatial/a9we-mtpn?method=export&format=Shapefile) - See it in action [here](https://data.cityofnewyork.us/Social-Services/NYC-Wi-Fi-Hotspot-Locations/a9we-mtpn)
 
##### Local File Alternatives for fall-back downloads
 * [nyc_boroughs.zip](https://web.dsa.missouri.edu/static/data/geodata/nyc_boroughs.zip)
 * [nyc_wifi.zip](https://web.dsa.missouri.edu/static/data/geodata/nyc_wifi.zip)
 

In [ ]:
import os
import pandas as pd
import numpy as np
from shapely.geometry import Point
from random import randint, uniform
import matplotlib.pyplot as plt

import urllib.request
import shutil
from pathlib import Path
from zipfile import ZipFile

import geopandas as gpd
import geoplot as gplt

In [ ]:
# Designate the URL for a file
file_URL = 'https://data.cityofnewyork.us/api/geospatial/tqmj-j8zm?method=export&format=Shapefile'

# Designate the local filename
local_file_name = 'nyc_boroughs.zip'

# Designate the local file name with a path to a temp directory.
     # Your Repo comes with this folder.  If not, use terminal and 
     # navigate to course folder > module3 and them : mkdir temp
file_Path = Path('../temp/')  
file_Path /= local_file_name

# Download the file from `url` and save it locally under `file_Path`:
with urllib.request.urlopen(file_URL) as response,  file_Path.open(mode='w+b') as out_file:
    shutil.copyfileobj(response, out_file)

# unzip file
to_unzip = ZipFile('../temp/nyc_boroughs.zip', 'r')
unzipped = '../temp/nyc_boroughs_unzipped'
to_unzip.extractall(unzipped)
to_unzip.close()
for file in os.listdir(unzipped):
    if file.endswith(".shp"):
        shape_file =  unzipped + '/' + file

In [ ]:
# Read data into GeoDataFrame and view the data
NYC = gpd.GeoDataFrame.from_file(shape_file)
NYC.head()

Let's do a quick plot to see what our data looks like

In [ ]:
# plot the data
NYC.plot()    

Now we are going to load the other data file. 

In [ ]:
# Designate the URL for a file
file_URL = 'https://data.cityofnewyork.us/api/geospatial/a9we-mtpn?method=export&format=Shapefile'

# Designate the local filename
local_file_name = 'nyc_wifi.zip'

# Designate the local file name with a path to a temp directory.
     # Your Repo comes with this folder.  If not, use terminal and 
     # navigate to course folder > module3 and them : mkdir temp
file_Path = Path('../temp/')  
file_Path /= local_file_name

# Download the file from `url` and save it locally under `file_Path`:
with urllib.request.urlopen(file_URL) as response,  file_Path.open(mode='w+b') as out_file:
    shutil.copyfileobj(response, out_file)

# unzip file
to_unzip = ZipFile('../temp/nyc_wifi.zip', 'r')
unzipped = os.getcwd() + '/nyc_wifi_unzipped'
to_unzip.extractall(unzipped)
to_unzip.close()
for file in os.listdir(unzipped):
    if file.endswith(".shp"):
        shape_file =  unzipped + '/' + file

# Read data into GeoDataFrame and view data
NYC_wifi = gpd.GeoDataFrame.from_file(shape_file)
NYC_wifi.head()

What does this data look like on a map?

In [ ]:
# plot the data
NYC_wifi.plot()  

Now that we have our data loaded, let's use some GeoPanda methods to examine and manipulate the geometry.

### Convex Hulls
Sometimes it is useful to be able to determine the smallest area that completely encloses a polygon, which is what convex hulls is. The following command will return a GeoSeries of geometries representing the smallest convex Polygon containing all the points in each object unless the number of points in the object is less than three. For two points, the convex hull collapses to a LineString; for 1, a Point.

In [ ]:
# create convex hulls
hulls = NYC['geometry'].convex_hull

# plot overlay
hulls.plot(ax=NYC.plot())

### Envelope
instead of the smallest of any shape, sometimes it is very useful to have the smallest rectangular polygon (with sides parallel to the coordinate axes) that contains each object. This is known as envelope, minimum bounding rectangle (MBR), or simply bounding box. The following command will return a GeoSeries of geometries representing the point or smallest rectangular polygon (with sides parallel to the coordinate axes) that contains each object.

In [ ]:
# create envelopes
envelope = gpd.GeoSeries(hulls.envelope)

# plot overlay
envelope.plot(ax=NYC.plot())

# you can also calculate the bounding boxes for each borough this way
boundingboxes = NYC['geometry'].bounds

print(boundingboxes)

## Centroid
Sometimes it is useful to summarize a polygon into a single point and, for that, a good candidate is its centroid (almost like a spatial analogue of the average). The following command will return a GeoSeries (a single column with spatial data) with the centroids of a polygon GeoDataFrame

In [ ]:
centriod = gpd.GeoSeries(NYC.centroid)
centriod.plot()

## Point in polygon (PiP)
Knowing whether a point is inside a polygon is conceptually a simple exercise but computationally a tricky task to perform. The simplest way to perform this operation in GeoPandas is through the contains method, available for each polygon object.

In [ ]:
Bronx = NYC[NYC['boro_name'] == 'Bronx']
Bronx.contains(centriod[0])

In [ ]:
Bronx.contains(centriod[2])

Performing point-in-polygon in this way is great to show the concept of point in poly, but for cases with many points and polygons, it is not particularly efficient. In these situations, it is much more advisable to perform then as a "spatial join". We will learn about these next week. 

## Buffers
Buffers are one of the classical GIS operations in which an area is drawn around a particular geometry, given a specific radius. These are very useful, for instance, in combination with point-in-polygon operations to calculate accessibility, catchment areas, etc.

To create a buffer using geopandas, simply call the buffer method, passing in the radius. Note that the radius needs to be specified in the same units as the CRS of the geography you are working with. 

In [ ]:
NYC_wifi.crs

This tells us it uses projection 4326 in the EPSG system. If we [look it up](http://epsg.io/4326), we will find that this corresponds with the World Geodetic System 1984 (GPS), which is expressed in degrees. Hence if we want, for example, a buffer of 2 degrees around each of the wifi points, we can simply obtain it by:

In [ ]:
buffer = NYC_wifi.buffer(.005)
buffer.head()

In [ ]:
f, ax = plt.subplots(1)
# Plot borough for reference
NYC.plot(ax=ax)
## Plot the wifi points
buffer.plot(ax=ax, color='red') 

plt.axis('equal')
plt.show()

## Area
Area is also very useful to know in regards to the data. Take a brief look at the area of the Bronx, according to wikipedia the area of land in the Bronx is 110 Square Kilometers. Let's see how we can calculate this. 

In [ ]:
Bronx = NYC[NYC['boro_name'] == 'Bronx']
Bronx.plot()

Once again we need to check our CRS to determine units.  

In [ ]:
Bronx.crs

Just like the NYC_wifi the projection is [4326](http://epsg.io/2831), which means the units are degrees. Let's change to a projection that has units in meters.


In [ ]:
#  Bronx = Bronx.to_crs({'init': 'epsg:2831'}) changed code to new version
Bronx = Bronx.to_crs('epsg:2831')

In [ ]:
area = (Bronx['geometry'].area / 1000000).iloc[0]

print(area)

Refer to [geopanda documentation](http://geopandas.org/geometric_manipulations.html) to see more operations that can be done. 

# Save your notebook

## The File > Save and Halt